# 2. Content-based model

In [1]:
# Давайте реализуем подобную рекомендательную систему на практике. 
# Будем работать с датасетом, содержащим информацию об оценивании фильмов на платформе Netflix.

import pandas as pd

df = pd.read_csv('data/netflix_titles.csv')
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [2]:
# В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. 
# Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. 
# Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход TF-IDF (Term Frequency-Inverse Document Frequency).

from sklearn.feature_extraction.text import TfidfVectorizer

# Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут содержательной информации, 
# и с учётом этого определим нашу модель:

model = TfidfVectorizer(stop_words='english')

#Заполним пропуски пустыми строками:

df['description'] = df['description'].fillna('')
#Трансформируем наши описания в матрицу:

feature_matrix = model.fit_transform(df['description'])

In [5]:
# 2.2
# Сколько столбцов в получившейся матрице?
feature_matrix.shape

(7787, 17905)

In [6]:
# Теперь необходимо вычислить косинусную близость. Можно сделать это так:

from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [7]:
# Вернём индексацию и уберём дубликаты из данных:

indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [9]:
# Теперь пропишем функцию для создания рекомендаций:

def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

get_recommendations("Star Trek")

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

In [10]:
# 2.3
# Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:

get_recommendations("Balto")

709                Balto 2: Wolf Quest
7446                           Vroomiz
1338    Chilling Adventures of Sabrina
7388                          Vampires
1770                          Dinotrux
2767                     Hold the Dark
5540                 Shanghai Fortress
4041                             Mercy
2582                       Half & Half
1365        Christmas in the Heartland
Name: title, dtype: object

# 3. Коллаборативная фильтрация

Мы рассмотрели несколько вариантов коллаборативной фильтрации на простейших примерах, и теперь пришло время практики с настоящими данными. Сначала мы будем использовать подход memory-based в модификации item-based, а затем SVD. В результате применения обоих алгоритмов мы сможем сравнить получившееся качество.  
  
Для создания алгоритмов рекомендательной системы будем использовать библиотеку surprise.

In [13]:
!pip3 install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 3.4 MB/s eta 0:00:00a 0:00:010m
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [14]:
!pip install scikit-surprise

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory


In [15]:
from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS #с помощью данного объекта мы можем использовать встроенные датасеты

data = Dataset.load_from_file(
    "data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

In [16]:
# Преобразуем данные к формату pandas DataFrame для удобной работы с ними:

df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

In [18]:
# 3.1
# Сколько уникальных фильмов в наборе данных?
df['movieId'].nunique()

1682

In [20]:
# 3.2
# Сколько уникальных пользователей в наборе данных?
df['userId'].nunique()

943

In [22]:
# 3.3
# Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.
df['rating'].mode()

0    4.0
Name: rating, dtype: float64

In [24]:
# 3.4
# Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. 
# В качестве значения параметра random_state возьмите число 13.

# Сколько объектов попало в тестовую выборку?
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25, random_state=13)
len(testset)

25000

In [25]:
# Импортируем функции для построения рекомендательных систем 
# (SVD — для model-based-подхода и KNNBasic — для memory-basic-подхода) и для оценки качества результата.

from surprise import SVD, KNNBasic, accuracy

In [26]:
# Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через косинусную близость и item-based-подход:

sim_options = {
    'name': 'cosine',
    'user_based': False
}
 
knn = KNNBasic(sim_options=sim_options)

In [27]:
# Обучим алгоритм:

knn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [28]:
# Теперь давайте посмотрим, какие рекомендации мы получили, с помощью следующей программы:

predictions = knn.test(testset)
predictions

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False}),
 Prediction(uid='804', iid='216', r_ui=4.0, est=3.922551907749182, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='350', iid='204', r_ui=4.0, est=4.345238219480267, details={'actual_k': 38, 'was_impossible': False}),
 Prediction(uid='708', iid='993', r_ui=4.0, est=3.4458505791534115, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='193', iid='1078', r_ui=4.0, es

In [36]:
# 3.5
# 1. Каков реальный рейтинг, выставленный пользователем с ID 500 для фильма с ID 699?

result = pd.DataFrame(predictions, columns=['uid', 'iid', 'r_ui', 'predict_est', 'details'])
result.drop(columns = {'details'}, inplace = True)
#result['erro'] = abs(result['base_event'] - result['predict_event'])
mask = result['uid']=='500'
result[mask]

,uid,iid,r_ui,predict_est
510,500,183,4.0,3.373462
760,500,557,3.0,3.250000
946,500,699,3.0,3.474790
1878,500,117,4.0,3.147919
1886,500,1057,3.0,3.150093
3283,500,25,3.0,3.423095
5012,500,582,4.0,3.318179
5337,500,313,3.0,3.350829
5501,500,721,1.0,3.324326
6064,500,405,4.0,3.073464


In [37]:
# Теперь необходимо вычислить RMSE для получившихся предсказаний:

accuracy.rmse(predictions)

RMSE: 1.0272


1.0271678039029761

In [38]:
# Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

# Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, 
# начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.

recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

In [40]:
# 3.6
# Реализуйте user-based-алгоритм. Какое значение RMSE получилось для коллаборативной фильтрации типа user-based? 
# Ответ округлите до двух знаков после точки-разделителя.

sim_options = {
    'name': 'cosine',
    'user_based': True
}
 
knn = KNNBasic(sim_options=sim_options)
knn.fit(trainset)
predictions = knn.test(testset)

accuracy.rmse(predictions)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.0174852296380237

In [41]:
# 3.7
# Теперь давайте сравним полученные результаты с результатами SVD-алгоритма. Реализуйте SVD с параметрами по умолчанию.

# Какое значение RMSE получилось для SVD? Ответ округлите до двух знаков после точки-разделителя.

svd = SVD()
svd.fit(trainset)
predictions = svd.test(testset)

accuracy.rmse(predictions)

RMSE: 0.9397


0.9397429194607837

# 4. Гибридные модели

In [42]:
!pip install lightfm

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 697.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-macosx_10_9_universal2.whl size=546320 sha256=6cdbcb2b04d05ad8b686606058dac3533269ba94e2799c69cde89db5536bdb3a
  Stored in directory: /Users/kseniamasnikova/Library/Caches/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [43]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k  

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [44]:
ratings = pd.read_csv('data/Gooddreadbooks/ratings.csv') # Поставленные оценки
books = pd.read_csv('data/Gooddreadbooks/books.csv') # Информация о книгах
tags = pd.read_csv('data/Gooddreadbooks/tags.csv') # Информация о тегах
book_tags = pd.read_csv('data/Gooddreadbooks/book_tags.csv') # Книги с тегами 

In [53]:
# 4.1
# Добавьте в набор данных book_tags признак с обычным id книги, используя соответствие обычного id и id в системе Goodreads.

# Какой обычный id у книги, которая имеет id 5 в системе Goodreads?

book_tags=book_tags.merge(books[['book_id','goodreads_book_id']],on='goodreads_book_id',how='left')
book_tags[book_tags['goodreads_book_id']==5]

,goodreads_book_id,tag_id,count,book_id
300,5,11557,40087,18
301,5,11305,39330,18
302,5,8717,17944,18
303,5,33114,12856,18
304,5,30574,11909,18
...,...,...,...,...
395,5,20781,299,18
396,5,32345,298,18
397,5,12600,282,18
398,5,3379,277,18


In [60]:
# 4.2
# Далее нам необходимо оставить в наборе данных book_tags только те записи, теги для которых есть в данных tags.

# Отфильтруйте данные таким образом, чтобы в наборе данных book_tags остались только те строки, 
# в которых находятся теги, информация о которых есть в наборе данных tags.

# Сколько объектов осталось?

tags_list = list(tags['tag_id'])
mask = book_tags['tag_id'].isin(tags_list)
book_tags=book_tags[mask]
book_tags

,goodreads_book_id,tag_id,count,book_id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27
6,1,14017,7169,27
10,1,27199,3857,27
...,...,...,...,...
999877,33288638,9886,10,8892
999879,33288638,3358,10,8892
999880,33288638,1679,10,8892
999889,33288638,1659,9,8892


In [61]:
# Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. 
# Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл ratings.

# Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. 
# Это можно сделать с помощью специальной функции из модуля scipy:

!pip install scipy==1.10
from scipy.sparse import csr_matrix

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 4.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.8.1
    Uninstalling scipy-1.8.1:
      Successfully uninstalled scipy-1.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
optuna 3.0.2 requires scipy<1.9.0,>=1.7.0, but you have scipy 1.10.0 which is incompatible.


In [62]:
# Осуществляем преобразование следующим образом:

ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) 
# Передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), 
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)

In [65]:
# Теперь нам необходимо составить матрицу с метаданными. 
# В качестве индексов будут выступать id книги и id тега, 
# и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id,book_tags.tag_id))) 

In [67]:
# 4.4
# Давайте проверим, что всё получилось правильно.

# Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

ratings_matrix.mean()

0.007086188900997592

In [68]:
# Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

model = LightFM(
    loss='warp-kos', # Определяем функцию потерь
    random_state=42, # Фиксируем случайное разбиение
    learning_rate=0.05, # Темп обучения
    no_components=100 # Размерность вектора для представления данных в модели
)

In [69]:
# Разобьём данные на обучающую и тестовую выборки:

train, test = random_train_test_split(
    ratings_matrix, # Общая выборка
    test_percentage=0.2, # Размер тестовой выборки
    random_state=42 # Генератор случайных чисел
)

In [70]:
# Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах. 
# Для этого воспользуемся методом fit(). В этот метод передадим обучающую выборку, признаки товаров — item_features, 
# количество эпох обучения (сколько раз мы будем показывать модели исходный датасет, чтобы она лучше выучила данные) — epochs, 
# а также параметр verbose для отслеживания процесса обучения:

model = model.fit(
    train, # Обучающая выборка
    item_features=meta_matrix, # Признаки товаров
    epochs=10, # Количество эпох
    verbose=True # Отображение обучения
)

Epoch: 100%|██████████| 10/10 [01:42<00:00, 10.25s/it]


In [72]:
# 4.5
# Оцените качество полученной модели с помощью функции precision_at_k, передав в неё три аргумента: 
# модель, тестовые данные и обозначение метаданных (item_features = meta_matrix).

# Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

precision = precision_at_k(model,test,item_features = meta_matrix)
precision.mean()

0.023578687

# 5. Современные методы: глубокое обучение

In [73]:
!pip install tensorflow

zsh:1: /usr/local/bin/pip: bad interpreter: /System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python: no such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 1.1 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 2.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 1.8 MB/s eta 0:00:00a 0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.4 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth

In [74]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [75]:
# Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, 
# выставленных книгам пользователями. Загрузим данные:

df = pd.read_csv('data/Gooddreadbooks/ratings.csv')

In [80]:
# Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра random_state возьмите число 42.

#Сколько объектов теперь находится в обучающей выборке?
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
print(train.shape)


(785404, 3)


In [83]:
# 5.2
# Запишите количество уникальных книг в переменную n_books.

# Сколько в наборе данных уникальных книг?

n_books = df['book_id'].nunique()
n_books

10000

In [85]:
# 5.3
# Запишите количество уникальных пользователей в переменную n_users.

# Сколько в наборе данных уникальных пользователей?
n_users = df['user_id'].nunique()
n_users

53424

In [87]:
# В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:
# Сначала мы задаём размерность входного слоя. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. 
# Далее мы разворачиваем результат в массив с одним измерением с помощью слоя Flatten().

book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

In [88]:
# Делаем то же самое для пользователей:

user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

In [89]:
# Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

conc = Concatenate()([book_vec, user_vec])

In [90]:
#Далее начинаем «собирать» нашу нейронную сеть из слоёв. Dense обозначает полносвязный слой. 
# Также мы обозначаем для него количество нейронов и данные, которые идут на вход.

fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

In [91]:
# Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

model2 = Model([user_input, book_input], out)

In [92]:
# Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. 
# В данном случае будем использовать метод adam и хорошо известную вам среднеквадратичную ошибку:

model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

In [93]:
# Теперь будем обучать нашу модель:

history = model.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

AttributeError: 'list' object has no attribute 'tocoo'

In [94]:
# Теперь можно оценить качество:

model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 2s 280us/step - loss: 15.8222


15.822196006774902

In [95]:
#Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. 
# Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 34s 1ms/step - loss: 0.8004
Epoch 2/10
24544/24544 [==============================] - 31s 1ms/step - loss: 0.6884
Epoch 3/10
24544/24544 [==============================] - 32s 1ms/step - loss: 0.6608
Epoch 4/10
24544/24544 [==============================] - 39s 2ms/step - loss: 0.6354
Epoch 5/10
24544/24544 [==============================] - 31s 1ms/step - loss: 0.6128
Epoch 6/10
24544/24544 [==============================] - 32s 1ms/step - loss: 0.5923
Epoch 7/10
24544/24544 [==============================] - 31s 1ms/step - loss: 0.5721
Epoch 8/10
24544/24544 [==============================] - 31s 1ms/step - loss: 0.5541
Epoch 9/10
24544/24544 [==============================] - 31s 1ms/step - loss: 0.5391
Epoch 10/10
6136/6136 [==============================] - 2s 285us/step - loss: 0.7444


0.744404673576355

# 6. Практика

In [96]:
shared = pd.read_csv('data/shared_articles.csv')
mask = shared['eventType']== 'CONTENT SHARED'
shared=shared[mask]
shared

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en
3,1459194474,CONTENT SHARED,-6151852268067518688,3891637997717104548,-1457532940883382585,NaN,NaN,NaN,HTML,https://cloudplatform.googleblog.com/2016/03/G...,Google Data Center 360° Tour,We're excited to share the Google Data Center ...,en
4,1459194497,CONTENT SHARED,2448026894306402386,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,https://bitcoinmagazine.com/articles/ibm-wants...,"IBM Wants to ""Evolve the Internet"" With Blockc...",The Aite Group projects the blockchain market ...,en
5,1459194522,CONTENT SHARED,-2826566343807132236,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.coindesk.com/ieee-blockchain-oxford...,IEEE to Talk Blockchain at Cloud Computing Oxf...,One of the largest and oldest organizations fo...,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3117,1487946604,CONTENT SHARED,9213260650272029784,3609194402293569455,7144190892417579456,Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebK...,SP,BR,HTML,https://startupi.com.br/2017/02/liga-ventures-...,"Conheça a Liga IoT, plataforma de inovação abe...","A Liga Ventures, aceleradora de startups espec...",pt
3118,1487947067,CONTENT SHARED,-3295913657316686039,6960073744377754728,-8193630595542572738,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3...,GA,US,HTML,https://thenextweb.com/apps/2017/02/14/amazon-...,Amazon takes on Skype and GoToMeeting with its...,"Amazon has launched Chime, a video conferencin...",en
3119,1488223224,CONTENT SHARED,3618271604906293310,1908339160857512799,-183341653743161643,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0...,SP,BR,HTML,https://code.org/about/2016,Code.org 2016 Annual Report,"February 9, 2017 - We begin each year with a l...",en
3120,1488300719,CONTENT SHARED,6607431762270322325,-1393866732742189886,2367029511384577082,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,MG,BR,HTML,https://www.bloomberg.com/news/articles/2017-0...,JPMorgan Software Does in Seconds What Took La...,"At JPMorgan Chase & Co., a learning machine is...",en


In [97]:
#  у нас есть информация о различных действиях пользователя, и на её основе мы должны создать некий универсальный индекс популярности. 
# Составим его из реакций пользователей, придав им разные веса:

event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [98]:
# Создайте признак, который будет отражать числовой вес для взаимодействия со статьёй (в соответствии с приведёнными выше весами). 
# Вычислите среднее значение для полученного признака. Округлите его до двух знаков после точки-разделителя.

users = pd.read_csv('data/users_interactions.csv')
users['v_event'] = users['eventType'].apply(lambda x: event_type[x])
users['v_event'].mean()

1.2362885828078327

In [99]:
# Чтобы получить хоть какую-то информацию, на которую можно будет опираться, оставьте только тех пользователей, 
# которые взаимодействовали хотя бы с пятью статьями. Сколько всего таких пользователей?

group = users.groupby(by='personId',as_index=False)['contentId'].value_counts()
person = group.groupby('personId')['contentId'].count().sort_values()
mask = person.values>=5
person_list = list(person[mask].index)
person[mask]

personId
-5179891559567091709      5
 7888432621799586316      5
-5444946123447491034      5
 202369545176637828       5
-3381527457560577265      5
                       ... 
-2979881261169775358    437
-1443636648652872475    585
-1032019229384696495    648
-2626634673110551643    669
 3609194402293569455    961
Name: contentId, Length: 1140, dtype: int64

In [100]:
# Теперь оставим только те взаимодействия, которые касаются только отфильтрованных пользователей 
# (то есть тех, которые взаимодействовали как минимум с пятью статьями). Сколько всего таких взаимодействий?
mask = users['personId'].isin(person_list)
users = users[mask]
users

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry,v_event
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN,1.0
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US,1.0
2,1465416190,VIEW,310515487419366995,-1130272294246983140,2631864456530402479,NaN,NaN,NaN,1.0
3,1465413895,FOLLOW,310515487419366995,344280948527967603,-3167637573980064150,NaN,NaN,NaN,3.0
4,1465412290,VIEW,-7820640624231356730,-445337111692715325,5611481178424124714,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...
72307,1485190425,LIKE,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,2.0
72308,1485190425,VIEW,-5813211845057621660,102305705598210278,5527770709392883642,Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/53...,SP,BR,1.0
72309,1485190072,VIEW,-1999468346928419252,-9196668942822132778,-8300596454915870873,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,SP,BR,1.0
72310,1485190434,VIEW,-6590819806697898649,-9016528795238256703,8614469745607949425,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4...,MG,BR,1.0


In [101]:
# Сейчас каждое отдельное взаимодействие пользователя со статьёй выделено в отдельную запись, 
# то есть пользователь мог просмотреть статью, лайкнуть и прокомментировать её, 
# и всё это отразилось в трёх действиях. 
# Давайте для удобства соединим все эти действия в некоторый коэффициент, который будет отражать интерес пользователя к статье. 
# Так как каждому возможному действию мы ранее уже присвоили вес, то, по сути, нам нужно просто сложить все действия. 
# Однако полученное число будет увеличиваться с количеством действий, и будет очень большой разброс возможных значений. 
# В таких случаях обычно логарифмируют полученный результат с помощью следующей функции:

import math

def smooth_user_preference(x):
    return math.log(1+x, 2)

In [102]:
# Примените упомянутое выше преобразование для логарифмирования к сумме весов для взаимодействия пользователя с каждой конкретной статьёй. 
# Также сохраните для каждой пары «пользователь — статья» значение времени последнего взаимодействия.

# Найдите среднее по признаку с получившимися временными отсечками. Округлите результат до двух знаков после точки-разделителя.

# Так как наши данные отсортированы по дате, то для того, чтобы выбрать последнее взаимодействие, необходимо использовать метод max().

data = users.groupby(by=['personId','contentId'], as_index=False)['v_event'].sum()
data['timestamp'] = users.groupby(by=['personId','contentId'], as_index=False)['timestamp'].max()['timestamp'] # добавляем столбец с последней датой
data['v_event'] = data['v_event'].apply(smooth_user_preference) # делаем логорифмирование
data['timestamp'].mean()

1470605340.0403006

In [103]:
# Разделите данные на обучающую и тестовую выборки, выбрав в качестве временной отсечки значение 1475519545. 
# Значение отсечки включите в тестовую выборку. Сколько объектов попало в обучающую выборку?

mask = data['timestamp']>= 1475519545
test = data[mask]
train =data.drop(test.index, axis=0)
train

,personId,contentId,v_event,timestamp
0,-9223121837663643404,-8949113594875411859,1.000000,1462452127
1,-9223121837663643404,-8377626164558006982,1.000000,1473938707
2,-9223121837663643404,-8208801367848627943,1.000000,1469706702
3,-9223121837663643404,-8187220755213888616,1.000000,1467823897
5,-9223121837663643404,-7331393944609614247,1.000000,1463486417
...,...,...,...,...
39087,9210530975708218054,4788854083489560153,1.584963,1466443715
39092,9210530975708218054,6527704900182256036,2.169925,1469621765
39093,9210530975708218054,6940370558848102253,1.000000,1472491975
39095,9210530975708218054,7434270606888620096,3.169925,1468265243


In [105]:
# Для удобства дальнейшего измерения качества рекомендаций преобразуйте данные так, чтобы получить таблицу в формате, 
# где строка соответствует пользователю, а столбцы будут истинными предпочтениями и рекомендациями в формате списков. 
# На место пустых ячеек поместите пустые списки.
import numpy as np

final_df = (
    train.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

final_df['true_test'] = (
    test.reset_index()
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

final_df['true_test'] = [ [] if x is np.NaN else x for x in final_df['true_test'] ]
final_df

,true_train,true_test
personId,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -..."
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[]
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084]
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]"
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -..."
...,...,...
9165571805999894845,"[-5488842573681626972, -3959242148361340089, -...",[]
9187866633451383747,"[-7660505434580831027, -5571606607344218289, -...","[-9039021554659316935, -4925213312961842798, -..."
9191849144618614467,"[-8742648016180281673, -8208801367848627943, -...","[-5605799891597699962, -4029704725707465084]"


In [106]:
# Осталось совсем немного — скоро вы получите свою первую систему рекомендаций! 
# Мы будем строить popular-based-модель, а значит, нам необходимо найти самые популярные статьи.

# Посчитайте популярность каждой статьи как сумму всех логарифмических «оценок» взаимодействий с ней (используя только обучающую выборку). 
# Выберите ID самой популярной статьи:

train.groupby('contentId')['v_event'].sum().sort_values(ascending=False)

contentId
-6783772548752091658    231.177195
-133139342397538859     228.024567
-8208801367848627943    189.937683
 8224860111193157980    186.044680
 7507067965574797372    179.094002
                           ...    
 1729892529360390310      1.000000
 1719976830095479814      1.000000
 7933362470375280252      1.000000
 6240076106289531207      1.000000
 9178250635927683         1.000000
Name: v_event, Length: 2366, dtype: float64

In [107]:
# Постройте систему рекомендаций. Оцените качество с помощью precision@10 для каждого пользователя (доля угаданных рекомендаций). 
# После этого усредните результат по всем пользователям.

recomend_content = list(train.groupby('contentId')['v_event'].sum().sort_values(ascending=False).head(10).index)
final_df['recomend'] = 'l'
for i in final_df.index:
    final_df['recomend'][i] = recomend_content

#Для вычисления precision@10 воспользуйтесь следующей функцией:

def precision(column):
    return (final_df.apply(lambda row:
            len(set(row['true_test']).intersection(set(row[column]))) /
            min(len(row['true_test']) + 0.001, 10.0),axis=1)).mean()
    
final_df['precision'] = precision('recomend')
final_df

,true_train,true_test,recomend,precision
personId,,,,
-9223121837663643404,"[-8949113594875411859, -8377626164558006982, -...","[-7423191370472335463, -6872546942144599345, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
-9212075797126931087,"[-1995591062742965408, -969155230116728853, 17...",[],"[-6783772548752091658, -133139342397538859, -8...",0.004286
-9207251133131336884,"[-9216926795620865886, -8742648016180281673, -...",[-4029704725707465084],"[-6783772548752091658, -133139342397538859, -8...",0.004286
-9199575329909162940,"[-5361115220834660562, -5002383425685129595, -...","[-3900870368325485697, 5037403311832115000]","[-6783772548752091658, -133139342397538859, -8...",0.004286
-9196668942822132778,[-721732705314803549],"[-8813724423497152538, -8535131855706279960, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
...,...,...,...,...
9165571805999894845,"[-5488842573681626972, -3959242148361340089, -...",[],"[-6783772548752091658, -133139342397538859, -8...",0.004286
9187866633451383747,"[-7660505434580831027, -5571606607344218289, -...","[-9039021554659316935, -4925213312961842798, -...","[-6783772548752091658, -133139342397538859, -8...",0.004286
9191849144618614467,"[-8742648016180281673, -8208801367848627943, -...","[-5605799891597699962, -4029704725707465084]","[-6783772548752091658, -133139342397538859, -8...",0.004286


## новые задания

Для начала необходимо построить матрицу, в которой по столбцам будут находиться id статей, по строкам — id пользователей, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

In [120]:
# 6.1
# Найдите оценку взаимодействия пользователя с ID -1032019229384696495 со статьёй с ID 943818026930898372. 
# Результат округлите до двух знаков после точки-разделителя.

# Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

ratings = pd.pivot_table(
    train,
    values="v_event",
    index="personId",
    columns="contentId",
).fillna(0)
ratings.loc[-1032019229384696495,943818026930898372]

2.321928094887362

In [125]:
# Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

# 6.2

# Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

r = ratings.values
r.mean()

0.016668620737604053

In [126]:
# 6.3
# Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. 
# Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). 
# Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.
similarity_users = np.zeros((len(r), len(r)))
for i in (range(len(r)-1)):
    for j in range(i+1, len(r)):
     
        mask_uv = (r[i] != 0) & (r[j] != 0)
        ratings_v = r[i, mask_uv]
        ratings_u = r[j, mask_uv]

        similarity_users[i,j] = np.corrcoef(ratings_v, ratings_u)[0, 1]
        similarity_users[j,i] = similarity_users[i,j]

similarity_users[3,40]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2845: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
/Library/Frameworks/Python.framework

nan